In [28]:
import os

from dotenv import load_dotenv
from markdown_it.rules_block import reference

print("Generating data")

Generating data


In [29]:
from dotenv import load_dotenv

load_dotenv()

True

In [30]:
import os
import pandas as pd

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [31]:
df = pd.read_csv('public/economy_delivery_data/real_data.csv')
df.head()

,Order ID,Customer ID,Platform,Order Date & Time,Delivery Time (Minutes),Product Category,Order Value (INR),Customer Feedback,Service Rating,Delivery Delay,Refund Requested
0,ORD000001,CUST2824,JioMart,19:29.5,30,Fruits & Vegetables,382,"Fast delivery, great service!",5,No,No
1,ORD000002,CUST1409,Blinkit,54:29.5,16,Dairy,279,Quick and reliable!,5,No,No
2,ORD000003,CUST5506,JioMart,21:29.5,25,Beverages,599,Items missing from order.,2,No,Yes
3,ORD000004,CUST5012,JioMart,19:29.5,42,Beverages,946,Items missing from order.,2,Yes,Yes
4,ORD000005,CUST4657,Blinkit,49:29.5,30,Beverages,334,"Fast delivery, great service!",5,No,No


In [32]:
import random
from faker import Faker

# Define the possible customer types
customer_types = ["Vip", "Bronze", "Sliver", "Gold", "Diamond"]

# Initialize Faker
fake = Faker('vi_VN')

# Create a dictionary to hold the fake data for each unique CustomerID
customer_info = {}

for cid in df['Customer ID'].unique():
    Faker.seed(random.randint(0, 100000))
    # Generate fake data
    country = fake.country()
    country_code = fake.country_code()
    # Note: Faker does not tie city to a specific country, so we simply generate a random city.
    city = fake.city()
    address = fake.address().replace("\n", ", ")  # Remove newlines for a single-line address.
    # Generate a fake name; if you want the name to reflect a specific locale based on the country,
    # you could create a new Faker instance with a locale (if available) for that country.
    name = fake.name()
    ctype = random.choice(customer_types)
    post_code = fake.postcode()

    # Save the fake data in the dictionary
    customer_info[cid] = {
        "CustomerCountry": country,
        "CustomerAddress": address,
        "CustomerCity": city,
        "CustomerName": name,
        "CustomerType": ctype,
        "CustomerPostCode": post_code,
        "CustomerCountryCode": country_code,
        "Currency": fake.currency_code(),
        "JobCustomer": fake.job(),
        "Phone": fake.phone_number(),
    }


In [33]:

list_company: list[str] = []
for _ in range(100):
    list_company.append(fake.company())

list_category: list[str] = ["Appliances", "Baby", "Wallets", "Beauty", "Fashion", "Mobile", "Laptop", "Tech", "Other", "Food"]


for oid in df['Order ID'].unique():
    Faker.seed(random.randint(0, 100000))

    order_price = fake.pricetag()
    order_payment_methods = random.choice(['Visa', 'MasterCard', 'Exchange', 'Money'])
    order_delivery_methods = random.choice(['Fast', 'Standard', 'VIP'])
    order_total_items = random.randint(1, 10)

    order_info[oid] = {
        'OrderValue': order_price,
        'OrderPaymentMethod': order_payment_methods,
        'OrderDeliveryMethod': order_delivery_methods,
        'OrderTotalItems': order_total_items,
        'productName': fake.domain_name(),
        'productInfo': fake.sentence(),
        'productId': fake.bothify('PROD#####'),
        'productBrand': random.choice(list_company),
        'productType': random.choice(list_category),
        "productImageUrl": fake.image_url(),
    }


In [34]:
customer_df = pd.DataFrame.from_dict(customer_info, orient='index').reset_index().rename(columns={'index': 'Customer ID'})

In [35]:
order_df = pd.DataFrame.from_dict(order_info, orient='index').reset_index().rename(columns={'index': 'Order ID'})

In [36]:
df = df.merge(customer_df, on='Customer ID', how='left')
df = df.merge(order_df, on='Order ID', how='left')

In [37]:
df.head()

,Order ID,Customer ID,Platform,Order Date & Time,Delivery Time (Minutes),Product Category,Order Value (INR),Customer Feedback,Service Rating,Delivery Delay,...,OrderValue,OrderPaymentMethod,OrderDeliveryMethod,OrderTotalItems,productName,productInfo,productId,productBrand,productType,productImageUrl
0,ORD000001,CUST2824,JioMart,19:29.5,30,Fruits & Vegetables,382,"Fast delivery, great service!",5,No,...,"₫89,405.34",Visa,Fast,9,pham.com,Các đến hơn chỉ này.,PROD58007,Đặng và Vũ Công ty TNHH MTV,Baby,https://picsum.photos/685/618
1,ORD000002,CUST1409,Blinkit,54:29.5,16,Dairy,279,Quick and reliable!,5,No,...,"₫9,936.57",Money,Fast,6,vu.com,Từng chỉ điều của mỗi.,PROD74791,Phạm và đối tác Doanh nghiệp tư nhân,Baby,https://placekitten.com/154/353
2,ORD000003,CUST5506,JioMart,21:29.5,25,Beverages,599,Items missing from order.,2,No,...,₫467.44,Money,Fast,8,vu.com,Dưới vì này cho có mà khi.,PROD94815,Phạm và đối tác Doanh nghiệp tư nhân,Tech,https://picsum.photos/438/58
3,ORD000004,CUST5012,JioMart,19:29.5,42,Beverages,946,Items missing from order.,2,Yes,...,"₫70,294.31",Money,VIP,5,mai.com,Chưa gần giống thì.,PROD19679,Trần và Mai Doanh nghiệp tư nhân,Laptop,https://picsum.photos/338/950
4,ORD000005,CUST4657,Blinkit,49:29.5,30,Beverages,334,"Fast delivery, great service!",5,No,...,₫461.30,Visa,Standard,4,le.com,Cũng trong đi cho bên vì nhưng như.,PROD22488,Hoàng và đối tác Công ty TNHH,Fashion,https://dummyimage.com/739x809


In [38]:
df.to_csv('public/economy_delivery_data/real_data_generate.csv', index=False)

In [39]:
# cut random 200 rows to become reference data
df1 = pd.read_csv('public/economy_delivery_data/real_data_generate.csv')
df1 = df1.sample(n=60, random_state=42)
df1.to_csv('public/economy_delivery_data/reference_data.csv', index=False)

In [40]:
reference_data = []
for col in df.columns:
    unique_values = df[col].dropna().unique()  # Remove NaN and get unique values
    if len(unique_values) > 50:
        continue
    unique_values_str = ", ".join(map(str, unique_values))  # Limit display to first 10 unique values
    reference_data.append({"column_name": col, "reference_value": unique_values_str})

reference_df = pd.DataFrame(reference_data)
reference_df.to_csv('public/economy_delivery_data/reference_data_value.csv', index=False)